**Задание**

Построить алгоритмы классификации для решения задачи классификации MNIST.
1. Обучить модели (SVM, logisitic regression, decision tree, (дополнительные модели по желанию) на исходных данных;
2. Использовать алгоритмы PCA и t-SNE для снижения размерности. Какое количество компонент необходимо, чтобы описать 90% дисперсии в PCA?
3. Обучить те же модели на сниженном признаковом пространстве, Какой алгоритм снижения дает лучшие метрики точности моделей классификации? Есть ли улучшение относительно работы моделей на исходном признаковом пространстве?
4. Оцените, какое количество времени требуется для препроцессинга PCA и t-SNE

In [4]:
# Импорт библиотек
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from time import time

In [5]:
# Загрузка датасета
df = load_digits()
X = df.data
y = df.target

# Введение параметров
random_state = 42
test_size = 0.2
max_iter=10000
n_neighbors=4

In [6]:
#Определение модели
models = {
    'SVM': SVC(random_state=random_state),
    'Logistic regression': LogisticRegression(random_state=random_state,max_iter=max_iter),
    'Decision tree': DecisionTreeClassifier(random_state=random_state),
    'kNN' :KNeighborsClassifier(n_neighbors=n_neighbors)
}

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    random_state=random_state)

In [22]:
for model_name, model_object in models.items():
    model_object.fit(X_train, y_train)
    y_predict = model_object.predict(X_test)
    print('\033[1m' + model_name +'\033[0m', '\n', classification_report(y_test, y_predict))

SVM 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       0.97      1.00      0.98        28
           2       1.00      0.94      0.97        33
           3       0.97      1.00      0.99        34
           4       1.00      1.00      1.00        46
           5       0.98      0.98      0.98        47
           6       0.97      1.00      0.99        35
           7       0.92      1.00      0.96        34
           8       0.94      0.97      0.95        30
           9       1.00      0.88      0.93        40

    accuracy                           0.97       360
   macro avg       0.97      0.98      0.97       360
weighted avg       0.98      0.97      0.97       360

Logistic regression 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       0.59      0.79      0.68        28
           2       0.84      0.94      0.89       

**Использование алгоритмов PCA и t-SNE для снижения размерности**

In [9]:
dimension_reduction_tools = {
    'PCA': PCA(n_components=0.90, random_state=random_state),
    't-SNE': TSNE(random_state=random_state)
}

X_processed = {}
processing_time = {}
for tool_name in dimension_reduction_tools.keys():
    start_time = time()

    try:
        dimension_reduction_tools[tool_name].fit(X)
        X_processed[tool_name] = dimension_reduction_tools[
            tool_name].transform(X)
    except AttributeError:
        X_processed[tool_name] = dimension_reduction_tools[
            tool_name].fit_transform(X)

    processing_time[tool_name] = time() - start_time

**Какое количество компонент необходимо, чтобы описать 90% дисперсии в PCA?**

In [10]:
print(dimension_reduction_tools['PCA'].n_components_)

21


**Обучение тех же моделей на сниженном признаковом пространстве**




In [21]:
for tool_name, tool in dimension_reduction_tools.items():
    print('\033[4m'+ tool_name+'\033[0m', '\n')

    X_train, X_test, y_train, y_test = train_test_split(
        X_processed[tool_name], y,
        test_size=test_size, random_state=random_state)

    for model_name, model_object in models.items():
        model_object.fit(X_train, y_train)
        y_predict = model_object.predict(X_test)
        print('\033[1m' + model_name +'\033[0m', '\n', classification_report(y_test, y_predict))

PCA 

SVM 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        34
           4       1.00      1.00      1.00        46
           5       0.98      0.98      0.98        47
           6       0.97      1.00      0.99        35
           7       0.97      0.97      0.97        34
           8       1.00      0.97      0.98        30
           9       0.95      0.95      0.95        40

    accuracy                           0.99       360
   macro avg       0.99      0.99      0.99       360
weighted avg       0.99      0.99      0.99       360

Logistic regression 
               precision    recall  f1-score   support

           0       0.94      0.97      0.96        33
           1       0.90      1.00      0.95        28
           2       0.97      0.94      0.95 

**Выводы**

* Для модели SVM одинаково хорошие результаты вышли при использовании алгоритма PCA. При этом алгоритм t-SNE выдал accuracy такой же как и до уменшения размерности.

* Для модели логистической регресии наилучшие результаты получились до уменешения размерности. При этом алгоритм PCA показал результат лучше,чем алгоритм t-SNE.

* Для дерева решений оба алгоритма дали улучшение относительно работы моделей на исходном признаковом пространстве.При этом алгоритм t-SNE показал результат лучше,чем алгоритм PCA.

* Для kNN модели одинаково хорошие вышли и до уменшения размерности, и при использовании обоих алгаритмов.

**Оценка количества времени, которое требуется для препроцессинга PCA и t-SNE**



In [12]:
for tool_name, tool_processing_time in processing_time.items():
    print(tool_name,round( tool_processing_time,3), 'с')

PCA 0.06 с
t-SNE 25.601 с
